<a href="https://colab.research.google.com/github/ByeonJaeseong/BOJprob/blob/main/Train_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install -U keras-tuner
!pip install bayesian-optimization

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from kerastuner.tuners import RandomSearch
from google.colab import files
import shutil
from tensorflow.keras.layers import Dropout
from bayes_opt import BayesianOptimization
from tensorflow.keras.layers import LayerNormalization
from google.colab import drive
drive.mount('/content/drive')
#uploaded = files.upload()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 사용자 정의 Weighted MAPE 손실 함수
def weighted_mape_loss(y_true, y_pred):
    # Calculate the absolute errors and absolute true values
    absolute_errors = tf.abs(y_true - y_pred)
    absolute_true = tf.abs(y_true)

    # Calculate the weights based on the time steps
    weights = tf.linspace(1.0001, 1.1999, tf.shape(y_true)[0])
    weights = tf.expand_dims(weights, axis=-1)  # Expand dimensions to match y_true

    # Calculate the weighted MAPE components
    epsilon = 1e-5  # Small constant to avoid division by zero
    weighted_mape_components = (weights * absolute_errors) / (absolute_true + epsilon)

    weighted_mape = tf.reduce_sum(weighted_mape_components)*100 / tf.cast(tf.shape(y_true)[0], tf.float32)
    return weighted_mape

In [52]:
list = ['s30', 's40', 's50', 's70', 's100', 'c30', 'c40', 'c50', 'c70', 'c100']
count = 1
# Define the hyperparameter tuning space and model architecture
def build_seq2seq_model(hp):
    encoder_lstm_units = hp.Int('encoder_lstm_units', min_value=16, max_value=128, step=8)
    decoder_lstm_units = hp.Int('decoder_lstm_units', min_value=16, max_value=128, step=8)
    num_layers = hp.Int('num_layers', min_value=1, max_value=4, step=1)
    optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop'])
    learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])

    # Define model inputs
    encoder_input = Input(shape=(X_features_train.shape[1],), name='encoder_input')
    decoder_input = Input(shape=(X_time_series_train.shape[1], 1), name='decoder_input')

    # Decoder
    decoder_output = decoder_input
    for _ in range(num_layers):
        decoder_output, state_h, state_c = LSTM(units=decoder_lstm_units, activation='tanh', return_sequences=True, return_state=True)(decoder_output)
        decoder_output = LayerNormalization()(decoder_output)

    output_sequence = Dense(4)(decoder_output)

    # Create the model
    model = Model(inputs=[encoder_input, decoder_input], outputs=output_sequence)
    model.compile(optimizer=optimizer, loss=weighted_mape_loss)
    return model





epochs = 100
batch_size = 32

for i in list:
    # 데이터 불러오기
    lane_data_c = pd.read_csv('/content/drive/MyDrive/train/' + 'lane_data_c.csv', encoding='utf-8')
    lane_data_s = pd.read_csv('/content/drive/MyDrive/train/' + 'lane_data_s.csv', encoding='utf-8')
    data = pd.read_csv('/content/drive/MyDrive/train/' + 'data_' + i + '.csv', encoding='utf-8')
    data_combined = pd.concat([lane_data_c, lane_data_s, data], axis=1)
    data_combined = data_combined.loc[:, ~data_combined.columns.duplicated()]
    data_combined = data_combined.drop_duplicates(subset='Distance', keep='first')

    # 입력 변수와 탈선계수 분리
    X_time_series = data_combined[['Distance']]
    X_features = data_combined.drop(['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2', 'Distance'], axis=1)
    y = data_combined[['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2']]

    # 데이터 정규화
    scaler = MinMaxScaler()
    X_features_scaled = scaler.fit_transform(X_features)

    # 학습 데이터와 테스트 데이터 분할
    X_time_series_train, X_time_series_test, X_features_train, X_features_test, y_train, y_test = train_test_split(X_time_series, X_features_scaled, y, test_size=0.2, random_state=42, shuffle=False)

    # 하이퍼파라미터 튜닝
    tuner = RandomSearch(build_seq2seq_model, objective='val_loss', max_trials=10, executions_per_trial=3, directory='tuner_results', project_name='seq2seq_tuning')
    tuner.search([X_features_train, X_time_series_train], y_train, epochs=epochs, batch_size=batch_size, validation_data=([X_features_test, X_time_series_test], y_test))

    best_model = tuner.get_best_models(num_models=1)[0]

    # 모델 학습
    model = build_seq2seq_model(hp)  # 최적의 하이퍼파라미터로 모델 구성
    model.fit([X_features_train, X_time_series_train], y_train, epochs=epochs, batch_size=batch_size, validation_data=([X_features_test, X_time_series_test], y_test))

    # 예측 및 결과 저장
    max_prediction_steps = 1999
    predicted_sequence = np.zeros((max_prediction_steps, 4))
    decoder_input = np.expand_dims(X_time_series_test.iloc[-1], axis=0)  # 마지막 테스트 데이터를 시작으로 예측

    for step in range(max_prediction_steps):
        prediction_step = best_model.predict([X_features_test[-1:], decoder_input])  # 이전 예측값과 특성을 입력으로 사용
        predicted_sequence[step] = prediction_step[0]

    # 이전 예측값을 3D 형태로 변환하여 다음 예측에 사용
    decoder_input = np.expand_dims(prediction_step, axis=1)  # 이번 예측값을 다음 입력으로 사용

    answer_sample = pd.read_csv('/content/drive/MyDrive/train/' + 'answer_sample.csv', header=None)
    answer_sample.iloc[1:, count:count+4] = predicted_sequence
    answer_sample.to_csv('/content/drive/MyDrive/train/' + 'answer_sample.csv', index=False, header=False)
    count = count + 4
    # 튜너 결과 삭제
    shutil.rmtree('tuner_results')


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
32                |32                |encoder_lstm_units
96                |96                |decoder_lstm_units
2                 |2                 |num_layers
rmsprop           |rmsprop           |optimizer
0.001             |0.001             |learning_rate

Epoch 1/100
202/300 [===================>..........] - ETA: 0s - loss: 1602280.1250

KeyboardInterrupt: ignored

In [53]:
# 튜너 결과 삭제
shutil.rmtree('tuner_results')